In [199]:
#####ベイジアン変量効果二項回帰モデル#####
##みどり本10章:個体差と生存種子数
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
from numpy.random import *
from scipy import optimize
from scipy.stats import norm


In [22]:
####データの発生####
#np.random.seed(8742)   #シードを設定

#データの設定
N = 500   #観測個体数
n = poisson(10, N)   #個体ごとの種子数

In [31]:
##応答変数の生成
#パラメータの設定
alpha = alphat = normal(0, 0.4, N)   #変量効果
beta = betat = -0.5   #全体共通の切片

#ロジットモデルと確率を設定
logit = alpha + beta   #ロジット
Pr = np.exp(logit) / (1 + np.exp(logit))   #確率

#二項分布から応答変数を生成
y = binomial(n, Pr, N)
pd.DataFrame({'1. 種子数' : n, '2. 生存数' : y, '3. 観測確率' : y/n, '4. 真の確率' : Pr})   #データの確認

,1. 種子数,2. 生存数,3. 観測確率,4. 真の確率
0,12,4,0.333333,0.341232
1,8,1,0.125000,0.400382
2,15,2,0.133333,0.257918
3,16,8,0.500000,0.318762
4,12,5,0.416667,0.177237
5,12,7,0.583333,0.466153
6,8,4,0.500000,0.430813
7,11,6,0.545455,0.260965
8,10,1,0.100000,0.325729
9,8,5,0.625000,0.562615


In [148]:
####マルコフ連鎖モンテカルロ法で変量効果二項回帰モデルを推定####
##二項回帰モデルの対数尤度関数を設定
def define_likelihood(alpha, beta, n, y):
    #ロジットと確率の定義
    logit = alpha + beta
    Pr = np.exp(logit)/(1+np.exp(logit))
    
    #対数尤度の計算
    LLi = y * np.log(Pr) + (n-y) * np.log(1-Pr)
    LL = sum(LLi)   
    return LL, LLi

In [149]:
##アルゴリズムの設定
R = 10000
keep = 4  
iter = 0
burnin = 2000/keep
disp = 10

##事前分布の設定
tau1 = 100   #パラメータの事前分布
s0 = 0.01   #階層モデルの事前分布
v0 = 0.01
rw1 = 0.1   #ランダムウォーク幅
rw2 = 0.15

##初期値の設定
alpha = normal(0, 0.2, N)
beta = 0
tau2 = 0.5   #階層モデルの分散の初期値

##パラメータの保存用配列
ALPHA = np.zeros((int(R/keep)))
BETA = np.zeros((int(R/keep)))
LL = np.zeros((int(R/keep)))

In [150]:
####メトロポリスヘイスティング法でパラメータをサンプリング####
##全体共通のbetaをサンプリング
#betaをサンプリング
betad = beta
betan = betad + normal(0, rw1)

#対数尤度と対数事前分布の計算
lognew1 = define_likelihood(alpha, betan, n, y)
logold1 = define_likelihood(alpha, betad, n, y)
logpnew1 = -0.5 * (np.power(betan, 2) / 2*tau1)
logpold1 = -0.5 * (np.power(betad, 2) / 2*tau1)

#MHサンプリングでbetaを採択するかどうかを決定
gamma = min(1, np.exp(lognew1[0] + logpnew1 - logold1[0] - logpold1))
u = uniform(0, 1, 1)

#alpha > u なら新しいbetaを採択
if gamma > u:
    beta = betan
    logl = lognew1
else :
    beta = betad
    logl = logold1

In [151]:
##変量効果alphaをサンプリング
#alphaをサンプリング
alphad = alpha
alphan = alphad + normal(0, rw2, N)

#対数尤度と対数事前分布の計算
lognew2 = define_likelihood(alpha, betan, n, y)
logold2 = logl
logpnew2 = -0.5 * (np.power(alphan, 2) / 2*tau2)
logpold2 = -0.5 * (np.power(alphad, 2) / 2*tau2)

#MHサンプリングでbetaを採択するかどうかを決定
rand = uniform(0, 1, N)   #一様分布から乱数を発生
LLind_diff = np.exp(lognew2[1] + logpnew2 - logold2[1] - logpold2)   #採択率
gamma = (LLind_diff > 1)*1 + (LLind_diff <= 1)*LLind_diff

#gammaの値に基づき新しいalphaを採択
flag = (gamma >= rand)*1 + (gamma < rand)*0
alpha = flag*alphan + (1-flag)*alphad

In [176]:
#逆ガンマ分布から階層モデルの分散をサンプリング
s = s0 + sum(np.power(alpha - np.mean(alpha), 2))   
v = v0 + N 
tau2 = np.sqrt(1/np.random.gamma(v/2, s/2))   #逆ガンマ分布からtauをサンプリング

In [192]:
#逆ガンマ分布から階層モデルの分散をサンプリング
s = s0 + sum(np.power(alphat - np.mean(alphat), 2))   
v = v0 + N 
tau2 = np.sqrt(1/np.random.gamma(v/2, s/2))   #逆ガンマ分布からtauをサンプリング

In [193]:
s = s0 + sum(np.power(alphat - np.mean(alphat), 2))   
v = v0 + N 
np.sqrt(1/np.random.gamma(v/2, s/2))   #逆ガンマ分布からtauをサンプリング

1.54042103437848e+143

In [198]:
np.random.gamma(v/2, s/2)


8931.525487598543

In [204]:
scipy.stats.gamma.rvs(v/2, s/2, 1)

302.341070168062

In [207]:
s

74.505704270656921